In [34]:
import pyspark
import findspark
import pymongo
from pymongo import MongoClient
from pyspark.sql import SparkSession

In [21]:
findspark.init()
sc = pyspark.SparkContext(appName="Spark Lab Session")

### Transforme Data to MongoDB

In [217]:
def extractActor1(array):
    actor1 = {}
    actor1['code']=array[5]
    actor1['name']=array[6]
    actor1['countryCode']=array[7]
    actor1['knownGroupCode']=array[8]
    actor1['ethnicCode']=array[9]
    
    actor1['religionCodes']=[array[10],array[11]]
    
    actor1['typeCodes']=[array[12],array[13],array[14]]
    
    geoActor1 = {}
    
    geoActor1['type']=array[35]
    geoActor1['fullname']=array[36]
    geoActor1['countryCode']=array[37]
    geoActor1['ADM1Code']=array[38]
    geoActor1['ADM2Code']=array[39]
    
    if(array[40]!='') and (array[41]!=''):
        geoActor1['coordinates']=[float(array[40]),float(array[41])]
    
    actor1['geography']=geoActor1
    
    
    return actor1   

In [218]:
def extractActor2(array):
    actor2 = {}
    actor2['code']=array[15]
    actor2['name']=array[16]
    actor2['countryCode']=array[17]
    actor2['knownGroupCode']=array[18]
    actor2['ethnicCode']=array[19]
    
    actor2['religionCodes']=[array[20],array[21]]
    
    actor2['typeCodes']=[array[22],array[23],array[24]]
    
    geoActor2 = {}
    
    geoActor2['type']=array[43]
    geoActor2['fullname']=array[44]
    geoActor2['countryCode']=array[45]
    geoActor2['ADM1Code']=array[46]
    geoActor2['ADM2Code']=array[47]
    
    if(array[48]!='') and (array[49]!=''):
        geoActor2['coordinates']=[float(array[48]),float(array[49])]
    
    actor2['geography']=geoActor2
    
    
    return actor2 

In [219]:
def extractActionGeo(array):
    actionGeo = {}
    actionGeo['type']=int(array[51])
    actionGeo['fullname']=array[52]
    actionGeo['countryCode']=array[53]
    actionGeo['ADM1Code']=array[54]
    actionGeo['ADM2Code']=array[55]
    
    if(array[56]!='') and (array[57]!=''):
        actionGeo['coordinates']=[float(array[56]),float(array[57])]
    
    return actionGeo

In [223]:
def convertExportRowToJSON(array):
    doc = {}
    doc['globalEventID']=int(array[0])
    doc['_id']=int(array[0])
    doc['day']=int(array[1])
    doc['monthyear']=int(array[2])
    doc['year']=int(array[3])
    
    if(array[4]!=''):
        doc['fractionDate']=float(array[4])
    
    doc['isRootEvent']=array[25]
    doc['eventCode']=array[26]
    doc['eventBaseCode']=array[27]
    doc['eventRootCode']=array[28]
    doc['quadClass']=int(array[29])
    
    if(array[30]!=''):
        doc['goldsteinScale']=float(array[30])
    
    doc['numMentions']=int(array[31])
    doc['numSources']=int(array[32])
    doc['numArticles']=int(array[33])
    
    if(array[34]!=''):
        doc['avgTone']=float(array[34])
        
    doc['dateadded']=array[59]
    doc['sourceurl']=array[60]
    
    doc['actor1']=extractActor1(array)
    doc['actor2']=extractActor2(array)
    doc['actionGeo']=extractActionGeo(array)
    return doc 

### Local Test

In [227]:
exportFile = '20200116144500.export.CSV'

rdd = sc.textFile(exportFile)\
        .map(lambda line: line.split("\t"))\
        .map(lambda array : convertExportRowToJSON(array))


In [228]:
rdd.take(1)

[{'globalEventID': 899488208,
  '_id': 899488208,
  'day': 20190116,
  'monthyear': 201901,
  'year': 2019,
  'fractionDate': 2019.0438,
  'isRootEvent': '0',
  'eventCode': '160',
  'eventBaseCode': '160',
  'eventRootCode': '16',
  'quadClass': 4,
  'goldsteinScale': -4.0,
  'numMentions': 8,
  'numSources': 1,
  'numArticles': 4,
  'avgTone': -3.51951032899771,
  'dateadded': '20200116144500',
  'sourceurl': 'https://www.dailymail.co.uk/news/article-7894935/Love-Island-Police-investigate-threats-Ollie-Williams-guards-protect-home-24-hours-day.html',
  'actor1': {'code': '',
   'name': '',
   'countryCode': '',
   'knownGroupCode': '',
   'ethnicCode': '',
   'religionCodes': ['', ''],
   'typeCodes': ['', '', ''],
   'geography': {'type': '0',
    'fullname': '',
    'countryCode': '',
    'ADM1Code': '',
    'ADM2Code': ''}},
  'actor2': {'code': 'MOZ',
   'name': 'MOZAMBIQUE',
   'countryCode': 'MOZ',
   'knownGroupCode': '',
   'ethnicCode': '',
   'religionCodes': ['', ''],
   '

In [229]:
client = MongoClient()
db = client.gdelt
db.events.drop()

In [230]:
#RDDtoMongoDB(rdd)

for row in rdd.collect():
    db.events.insert_one(row)

In [231]:
list(db.events.find({'actor1.countryCode':''}).limit(10))

[{'_id': 899488208,
  'globalEventID': 899488208,
  'day': 20190116,
  'monthyear': 201901,
  'year': 2019,
  'fractionDate': 2019.0438,
  'isRootEvent': '0',
  'eventCode': '160',
  'eventBaseCode': '160',
  'eventRootCode': '16',
  'quadClass': 4,
  'goldsteinScale': -4.0,
  'numMentions': 8,
  'numSources': 1,
  'numArticles': 4,
  'avgTone': -3.51951032899771,
  'dateadded': '20200116144500',
  'sourceurl': 'https://www.dailymail.co.uk/news/article-7894935/Love-Island-Police-investigate-threats-Ollie-Williams-guards-protect-home-24-hours-day.html',
  'actor1': {'code': '',
   'name': '',
   'countryCode': '',
   'knownGroupCode': '',
   'ethnicCode': '',
   'religionCodes': ['', ''],
   'typeCodes': ['', '', ''],
   'geography': {'type': '0',
    'fullname': '',
    'countryCode': '',
    'ADM1Code': '',
    'ADM2Code': ''}},
  'actor2': {'code': 'MOZ',
   'name': 'MOZAMBIQUE',
   'countryCode': 'MOZ',
   'knownGroupCode': '',
   'ethnicCode': '',
   'religionCodes': ['', ''],
   '